In [152]:
import pandas as pd
from config import gkey
from config import gkey2
import numpy as np
import requests
import json
import geopy.distance as gp

# Beginning the data-gathering process

I found a PDF provided by the U.S. Department of Education identifying Title 1 schools and their respective poverty levels. From this I created a csv of school names, addresses, and poverty level in Austin, Dallas, and Laredo. 

Using the csv I made, I then used Google's geolocation API to get each school's latitude and longitude. We now had a starting point for distance to grocery stores.

To find nearby grocery stores, I decided to use the Google nearby search API to look up grocery stores within a 10-kilometer radius of the center of Austin by the type 'supermarket'. My rationale for taking this approach as opposed to going school-to-school and doing a nearby search was that I could quickly grab all grocery stores in the area and then use geopy to compare school coordinates to the grocery store coordinates to find the closest sets. This worked, but turned out not to be as simple as it looked. There was a lot of data cleaning required before I could run geopy.

# Setting up the geocoordinates API

In [44]:
# Using Austin's geocoordinates, I set up the API call

Austin_lat = 30.267153
Austin_long = -97.7430608 
lat_long = [Austin_lat,Austin_long]

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"key": gkey, 
          "location": f"{Austin_lat},{Austin_long}", 
          "radius": 10000, 
          "type": "supermarket"}

data = requests.get(base_url, params = params).json()

I created a series of arrays to hold output. This loop uses i as a counter and runs through the first page of Google returns. I used the next_page_token (not shown here) when results outputted multiple pages. 

In [46]:
name = []
lat = []
long = []
address = []

i = 0

while i < 20:
    
    name.append(data["results"][i]["name"]),
    lat.append(data["results"][i]["geometry"]["location"]["lat"]),
    long.append(data["results"][i]["geometry"]["location"]["lng"]),
    address.append(data["results"][i]["vicinity"])
    
    i += 1

# Something is amiss...

A quick look at the first 10 entries in the list 'name' shows that HEB and Randalls are both missing. Moreover, Family Dollar, a store that doesn't meet our criteria of having fresh fruits and vegetables available, needed to be cleaned from the dataset. 

In [48]:
name[0:10]

["Trader Joe's",
 'Walmart Supercenter',
 'Wheatsville Food Co-Op',
 'Walmart Supercenter',
 'Fiesta Mart',
 'Walmart Supercenter',
 'Fiesta Mart',
 'Family Dollar',
 'Family Dollar',
 'Walmart Supercenter']

To find HEB and, later, Randalls stores in Austin, I used the same procedure as above, but instead of calling "type": "supermarket", I used the store name to find matches.

In [4]:
params = {"key": gkey, 
          "location": f"{Austin_lat},{Austin_long}", 
          "radius": 10000, 
          "name": "HEB"}

Checking the data assured me that I was on the right track.

In [17]:
data["results"][0]["name"]

'H-E-B'

Again, I created a series of arrays to hold output and dropped them into a dataframe.

In [34]:
df = pd.DataFrame()
df["Store Name"] = name
df["Store Address"] = address
df["Lat"] = lat
df["Long"] = long

# Duplicates
The head looks good, but when you get to the tail, you see that some stores show up multiple times. Almost always, it was a bakery within a supermarket. Somehow the bakery has its own unique geocoordinates. 

In [36]:
df.tail(2)

,Store Name,Store Address,Lat,Long
18,H-E-B Bakery,West Lake Hills,30.291743,-97.824836
19,H-E-B Bakery,"2400 S Congress Ave, Austin",30.238694,-97.754756


In [37]:
df = df.drop_duplicates(subset = ["Store Address"])

Getting rid of duplicates by address was helpful, but not all bakeries had the full address listed.

In [38]:
df.tail(3)

,Store Name,Store Address,Lat,Long
13,H-E-B,"2110 W Slaughter Ln, Austin",30.175379,-97.825078
16,H-E-B Bakery,Austin,30.216230,-97.831086
18,H-E-B Bakery,West Lake Hills,30.291743,-97.824836


A quick look at the entire dataframe showed that stores 11, 16, and 18 are duplicates. I dropped those by index.

In [39]:
df

,Store Name,Store Address,Lat,Long
0,H-E-B,"6900 Brodie Ln, Austin",30.216283,-97.830987
1,H-E-B,"600 W William Cannon Dr, Austin",30.197923,-97.786481
2,H-E-B,"2400 S Congress Ave, Austin",30.238729,-97.755227
3,H-E-B,"6607 S IH 35 Frontage Rd, Austin",30.188724,-97.768718
4,H-E-B,"701 N Capital of Texas Hwy Bld C, West Lake Hills",30.291796,-97.825190
5,H-E-B,"1000 E 41st St, Austin",30.300637,-97.719957
6,H-E-B plus!,"2508 E Riverside Dr, Austin",30.236523,-97.722073
7,H-E-B,"2701 E 7th St, Austin",30.259124,-97.711619
8,H-E-B,"5808 Burnet Rd, Austin",30.334406,-97.741126
9,H-E-B,"1801 E 51st St, Austin",30.301166,-97.698715


In [40]:
df = df.drop(11)
df = df.drop(16)
df = df.drop(18)

The HEB data for Austin is ready to go. I used the same process to get clean data for Randalls stores and then combined the two with cleaned data from the Google API.

In [43]:
# Clean HEB data for Austin

df

,Store Name,Store Address,Lat,Long
0,H-E-B,"6900 Brodie Ln, Austin",30.216283,-97.830987
1,H-E-B,"600 W William Cannon Dr, Austin",30.197923,-97.786481
2,H-E-B,"2400 S Congress Ave, Austin",30.238729,-97.755227
3,H-E-B,"6607 S IH 35 Frontage Rd, Austin",30.188724,-97.768718
4,H-E-B,"701 N Capital of Texas Hwy Bld C, West Lake Hills",30.291796,-97.825190
5,H-E-B,"1000 E 41st St, Austin",30.300637,-97.719957
6,H-E-B plus!,"2508 E Riverside Dr, Austin",30.236523,-97.722073
7,H-E-B,"2701 E 7th St, Austin",30.259124,-97.711619
8,H-E-B,"5808 Burnet Rd, Austin",30.334406,-97.741126
9,H-E-B,"1801 E 51st St, Austin",30.301166,-97.698715


# Find the distance between two points
For the following example, I've imported previously cleaned school data and grocery store data.

In [134]:
infile = pd.read_csv("Austin_Coords.csv")
school_df = pd.DataFrame(infile)
school_df.head(2)

,School Name,Location,Percent in Poverty,Lat,Long,Google Place ID
0,Allison Elementary,Austin,92.00,30.168207,-97.817760,ChIJ04RBdWa3RIYRgJ7aGK5dgXM
1,Andrews Elementary School,Austin,91.02,30.317554,-97.679663,ChIJ6YaMBenJRIYRhrPJHKXhGBU


In [118]:
infile = pd.read_csv("Austin_groceries.csv")
grocery_df = pd.DataFrame(infile)
grocery_df.head(2)

,Store Name,Location,Lat,Long,Vicinity
0,H-E-B,Austin,30.216284,-97.830988,"6900 Brodie Ln, Austin"
1,H-E-B,Austin,30.197923,-97.786481,"600 W William Cannon Dr, Austin"


I needed arrays for lat/long, so I set those up.

In [65]:
store_lat, store_long = grocery_df["Lat"], grocery_df["Long"]
school_lat, school_long = school_df["Lat"], school_df["Long"]

I tested the first two sets of coordinates to see if the geopy distance function worked. It did.

In [66]:
gp.distance(f'{school_lat[0]},{school_long[0]}', f'{store_lat[0]},{store_long[0]}').miles

3.4049041064242647

# Identifying which store matches which school

First I needed arrays for the school and store names, both to find the length of the loop and to compare the names against the indices the loops would output.

In [ ]:
schools = school_df["School Name"]
stores = grocery_df["Store Name"]

Next I created an array to hold the closest store's distance and to hold the index for the store. 

In [188]:
closest_store_distance = []
closest_store_index_list = []

I set up a way to keep track of the index for each school and each store. The indices would correspond to counters within each loop. The school index would increment each time the outer loop ran, and the store index would match the counter on the inner loop whenever it hit the condition of being the closest store.

This was the key to locating the store that corresponded to the appropriate school. 

In [ ]:
school_index = 0
store_index = 0

The outer school loop runs the first school against every grocery store. So school [0] runs against store [0], then store [1], store [2], etc. By making the counter j increment as each store iterates through the inner loop, the counter stays the same as the dataframe index of each store. Counter i in the outside loop corresponds to the school index, as they both increment by one as each school is looped through.

In [189]:
i = 0
j = 0

I set up a temporary variable called closest. This holds the smallest distance as each store is compared to a school. Whenever a smaller distance appears, closest is updated. Whenever that update happens, the store index is set as the counter j, which matches that store's index in the dataframe.

I set closest equal to 10 because geopy is using miles as a measurement. The closest store will always be closer than 10 miles (in this dataset, at least) and so closest will be updated the first time a store is less than 10 miles away and then updated again as stores get closer.

This process is a brute force method--which is really inefficient--but since this is a small dataset, it worked fine and ran quickly.

closest = 10

Two while loops ran the length of the schools and stores arrays. Geopy distance grabbed the distance between two points, then the if loop compared it to the closest variable, recording it and the corresponding counter when it was closer than the value already stored. As each inner loop finished, the store index was appended to the closest_store_index_list array and its corresponding distance to the closest_store_distance array.

In [137]:
while i < len(schools):
    
    while j < len(stores):
        
        distance = gp.distance(f'{school_lat[i]},{school_long[i]}', 
                               f'{store_lat[j]},{store_long[j]}').miles
        
        if distance < closest:
            closest = distance
            store_index = j
        
        # incrementing j moves us to the next store in the array
            
        j += 1
            
    # Now that the initial loop is finished, I append the distance and the correct indices,
    # then reset the j index so that the store indices can be looped through correctly again
    
    closest_store_distance.append(closest)
    closest_store_index_list.append(store_index)
    
    # Temporary variable and counter are reset for the next loop
    
    closest = 10
    j = 0
    
    # Now the school index will be incremented so that we loop through all schools
    
    i += 1
    

Checking the first five outputs for each list assures that they look as expected

In [95]:
closest_store_index_list[:5]

[12, 11, 2, 8, 7]

In [138]:
closest_store_distance[:5]

[0.6602052745641682,
 0.9677309053194558,
 0.8419731374081209,
 0.8305586639033838,
 0.6254233778464495]

# Now we need a dataframe with schools and stores correctly aligned
To prepare a new dataframe that matches schools with closest stores, I created arrays to hold the appropriate store information.

In [96]:
close_store = []
store_address = []

By looping through closest_store_index_list, I got the index I needed to pull store info out of the grocery_df dataframe.

In [119]:
for store in closest_store_index_list:
    
    close_store.append(grocery_df.iloc[store]["Store Name"])
    store_address.append(grocery_df.iloc[store]["Vicinity"])

I created an updated dataframe to hold school name and location, store name and address, the distance between the school and the store, and the percent of kids at the school who fall below the poverty threshold. Since the schools stayed in order (only the grocery store indexes moved around), I pulled the school name, location, and poverty information straight from school_df.

Note: Because I'd already combined some datasets before putting the example into this notebook, I had to truncate the close_store list below.

In [147]:
updated_df = pd.DataFrame()
updated_df["School Name"] = school_df["School Name"]
updated_df["Location"] = school_df["Location"]
updated_df["Closest Store"] = close_store[:61]
updated_df["Store Address"] = store_address
updated_df["Distance in Miles"] = closest_store_distance
updated_df["Percent in Poverty"] = school_df["Percent in Poverty"]

In [148]:
updated_df.head(3)

,School Name,Location,Closest Store,Store Address,Distance in Miles,Percent in Poverty
0,Allison Elementary,Austin,H-E-B,"2110 W Slaughter Ln, Austin",0.660205,92.00
1,Andrews Elementary School,Austin,H-E-B,"7112 Ed Bluestein Blvd #125, Austin",0.967731,91.02
2,Becker Elementary School,Austin,H-E-B,"2400 S Congress Ave, Austin",0.841973,64.52


To ensure that the data was accurate, I hand-checked a few randomly chosen schools against the corresponding stores to make sure they matched. 

# There's got to be a better way!
Doing all of the data processing above proved to be so time-consuming that Kellye and I decided to try using the Google nearby search API for each school in Dallas and Laredo. All of the problems we experienced before (major grocery chains missing, wrong types popping up) were now individualized, which just made data collection more time-consuming. After a while decided to find the nearest grocery store by looking at the schools on Google Maps, searching for stores nearby, and then picking the closest one that met our criteria. Time-wise, this was only slightly better -- but at least it was less frustrating.

# When is close close enough?

The geometric distance that geopy produced is point-to-point. We almost never travel that way in real life. I wanted to see if we could get a more accurate distance. Turns out we can: the Google Distance Matrix API will pull driving distance between two points.

For this example, I don't have a dataframe containing the school address. Since the schools have stayed in the same order, I can use the school_lat and school_long arrays made earlier as the origin points. Because the stores have been re-ordered, however, it's best to pull the store addresses from updated_df into a new array.

In [174]:
store_address = updated_df["Store Address"]
store_address[0]

'2110 W Slaughter Ln, Austin'

With a new API, we have to get a new base url. It turns out I also had to get an additional API key; the one for geolocation and nearby searches doesn't work for this API.

In [149]:
# New base url

base_url = "https://maps.googleapis.com/maps/api/distancematrix/json?"

A helpful thing to know about the Distance Matrix API is that it outputs distance as a string with its units of measure attached.

In [169]:
data["rows"][0]["elements"][0]["distance"]["text"]

'1.0 mi'

Since I wanted to be able to use the distance as a floating point number for calculations, I added a few lines of code to the while loop below that stripped off the non-numeric characters and then cast the result as a floating point number before appending it to the driving_distance array.

In [167]:
i = 0

# To double-check addresses in the output match those in the input, I created arrays to hold the 
# results from the API pull

dest_address = []
orig_address = []

# The driving distance array gave the new information I wanted.

driving_distance = []

while i < len(schools):
    
    params = {
    "origins": f"{school_lat[i]},{school_long[i]}", 
    "key": gkey2, 
    "destinations": store_address[i], 
    "units": "imperial"}
    
    data = requests.get(base_url, params = params).json()
    
    dest_address.append(data["destination_addresses"][0])
    orig_address.append(data["origin_addresses"][0])
    
    temp = (data["rows"][0]["elements"][0]["distance"]["text"])
    temp = temp[:-3]
    driving_distance.append(float(temp))
    
    break
    i += 1


Checking the addresses pulled by the API allowed me to double-check that I had the right origin and destination. I had school addresses in a dataframe not listed here, so I was able to use those for comparison, and the outputted addresses matched the input. I would have preferred to output the school name, but the Distance Matrix API does not include a name in its results -- only addresses and distances.

You can see below that the H-E-B addresses in the output and in the existing dataframe match.

In [168]:
print(orig_address, dest_address, driving_distance)

['10113 Aly May Dr, Austin, TX 78748, USA'] ['2110 W Slaughter Ln, Austin, TX 78748, USA'] [1.0]


In [178]:
updated_df.head(1)

,School Name,Location,Closest Store,Store Address,Distance in Miles,Percent in Poverty
0,Allison Elementary,Austin,H-E-B,"2110 W Slaughter Ln, Austin",0.660205,92.0


The outputted driving distance for Allison Elementary is significantly longer than the point-to-point 
distance. This school barely made it into the 'not a food desert' category. 
Had the distance been greater than one mile, it would have qualified as a food desert. 
Many other schools had a status change when the driving distance turned out to be longer 
a mile.

In [185]:
print(f'Driving distance was {driving_distance[0]} miles')
print(f'Point-to-point distance was {updated_df["Distance in Miles"][0]} miles')

Driving distance was 1.0 miles
Point-to-point distance was 0.6602052745641682 miles


# Merging Dataframes
Once I had all of the data I needed from each city, I combined them into one dataframe. I left the store address in the combined dataframe because I wanted to know how many grocery stores we were working with, and store name is not a unique identifier.

In [187]:
infile = pd.read_csv("Combined_Cities.csv")
df = pd.DataFrame(infile)
df.nunique()

Bus Distance (miles)         46
Closest Store                33
Driving Distance (miles)     40
Location                      3
Percent in Poverty          187
Poverty Groups                7
School Name                 219
Store Address                94
Walking Distance (miles)     34
Zip Code                     58
Food Desert                   2
dtype: int64

As you can see, we had 33 unique names for closest store, but 94 unique addresses -- so there were 94 grocery stores in the dataset. I also found the walking and bus-riding distances for some cities using the Distance Matrix API, but we ended up not using them in the data analysis.